# Mu Numerical Residuals

This notebook demonstrates residuals between $\mu$ values calculated by `astopy` and `sncosmo`.

In [ ]:
from pathlib import Path

import numpy as np
import sncosmo
from astropy.cosmology import WMAP9, FlatLambdaCDM
from matplotlib import pyplot as plt

from scipy.optimize import minimize 


In [ ]:
def apparent_mag_for_model(model):
    """Calculate the distance modulus of a model

    Args:
        model (Model): An sncosmo model

    Returns:
        mu = m_B - M_B
    """

    dilation_factor = 1 + model['z']
    time_dilation_mag_offset = - 2.5 * np.log10(dilation_factor)

    b_band = sncosmo.get_bandpass('standard::b')
    rest_band = b_band.shifted(dilation_factor)

    apparent_mag = model.bandmag(rest_band, 'ab', 0) - time_dilation_mag_offset
    return apparent_mag


To demonstrate the effect, we set the peak absolute brightness of a model, calculate the apparent magnitude for that model, and determine $\mu$ as the difference of those two values. 

In [ ]:
model = sncosmo.Model('salt2-extended')
z_vals = np.arange(.01, 1.1, .01)
abs_mag = -19.05

apparent_mag = []
absolute_mag = []
for z in z_vals:
    model.set(z=z)
    model.set_source_peakabsmag(abs_mag, 'standard::b', 'AB')
    apparent_mag.append(apparent_mag_for_model(model))
    
apparent_mag = np.array(apparent_mag)
mu = apparent_mag - abs_mag


In [ ]:
plt.plot(z_vals, mu - WMAP9.distmod(z_vals).value)
plt.xlabel('z', fontsize=12)
plt.ylabel(r'$\Delta\mu$ (sncosmo - WMAP9)', fontsize=12)


One potential source is the interpolation used by `sncosmo` when setting the absolute magnitude of the model. We see below that this isn't the case.

In [ ]:
def calc_abs_mag_offset(
    cosmo, z, abs_mb=-19.05, band='standard::b', magsys='AB'):
    
    source = 'salt2-extended'
    model = sncosmo.Model(source)
    model.set(z=z)
    model.set_source_peakabsmag(abs_mb, band, magsys, cosmo=cosmo)
    
    return abs_mb - model.source_peakabsmag(band, 'AB', cosmo=cosmo)


In [ ]:
z_vals = np.arange(.01, 1.1, .01)
wmap_mag_offset = [calc_abs_mag_offset(WMAP9, z) for z in z_vals]
plt.plot(z_vals, wmap_mag_offset)
